In [ ]:
#backtester

import math
import pandas as pd
import pandas_ta as ta 
import numpy as np
from backtesting import Strategy 

class SimpleRSI(Strategy):
    #tunable params:
    RSI_PERIOD   = 14
    RSI_ENTRY    = 30       # cross UP through this -> buy
    RSI_EXIT     = 70       # cross DOWN through this -> exit
    SMA_TREND    = 200      # only trade if Close > SMA(200)
    ATR_PERIOD   = 14
    ATR_MULT     = 2.0      # conservative stop = Entry - ATR_MULT * ATR (average true range)
    PER_TRADE_CAP = 0.10    # fraction of available cash to commit (0.0–1.0)

    def init(self):
        close = self.data.Close #most recent price
        #rsi indicator set up
        self.rsi = self.I(lambda x: ta.rsi(x, legnth=self.RSI_PERIOD), close)
        #sma indicator set up
        self.sma = self.I(lambda x: x.rolling(self.SMA_TREND).mean(), close)
        #average true range for stop loss sizeing (using tuple unpacking)
        high, low = self.data.High, self.data.Low
        self.atr = self.I(
            lambda h, l, c: ta.atr(
                h, l, c, length=self.ATR_PERIOD),
                high, low, close)
        
        @staticmethod #<- bypass self.
        def _cross_up(series, level)-> bool:
            ''' yesterday below level and today at/above level'''
            return len(series) >= 2 and series[-2] < level <= series[-1]
        
        @staticmethod #<- bypass self.
        def _cross_down(series, level)-> bool: 
            return len(series) >= 2 and series[-2] > level >= series[-1]
        
        def next(self):
            price = self.data.Close[-1]
            above_trend = price > self.sma[-1] if pd.notna(self.sma[-1]) else False

In [ ]:
# data.py 

